# SQL in Python

- hide: true
- toc: true
- comments: true
- categories: [python, SQL]

## SQLite

Based on [docs](https://docs.python.org/3/library/sqlite3.html).

In [22]:
import os
import sqlite3

PATH = '/Users/fgu/tmp'

sqlite3.version

'2.6.0'

Establish a connection to database (or create and connect if it doesn't exist yet)

In [23]:
db_path = os.path.join(PATH, 'some.db')

conn = sqlite3.connect(db_path)

To manupulate the database, create a [cursor](https://en.wikipedia.org/wiki/Cursor_(databases)) object (not strictly [needed](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.execute) for `sqlite3`), but I still use it for practice and to make db code portable to other databases.

### Adding a table with a row of data

In [ ]:
# Create cursor
c = conn.cursor()

# Create table
c.execute("""CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)""")

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) changes
conn.commit()

# Close connection
conn.close()

To check that the database now contains our stocks table, list all its tables.

In [25]:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("select name from sqlite_master where type = 'table'").fetchall()

[('stocks',)]

### Retrieving data

In [26]:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT * FROM stocks").fetchall()

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]

When adding Python variables to the query, never use string substitution directly like so:

In [27]:
symbol = 'RHAT'
c.execute(f"select * from stocks where symbol = '{symbol}'").fetchall()

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]

While this works, it's vulnerable to [injection attacks](https://xkcd.com/327/). Use parameter substition instead. Either using question marks like so

In [28]:
symbol = ('RHAT', )
c.execute("select * from stocks where symbol = ?", symbol).fetchall()

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]

or using named placedholders like so

In [29]:
c.execute("select * from stocks where symbol = :symbol", {'symbol': 'RHAT'}).fetchall()

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]

### Why do I need `fetchall()` after `cursor.execute()`?

Because the `curse.execute()` returns an iterater object containing all query results.

### Using namedtuples

In [ ]:
from collections import namedtuple

EmployeeRecord = namedtuple('EmployeeRecord', 'name, age, title, department, paygrade')

import csv
for emp in map(EmployeeRecord._make, csv.reader(open("data/employees.csv", "rb"))):
    print (emp.name, emp.title)

# import sqlite3
# conn = sqlite3.connect('data/companydata')
# cursor = conn.cursor()
# cursor.execute('SELECT name, age, title, department, paygrade FROM employees')
# for emp in map(EmployeeRecord._make, cursor.fetchall()):
#     print(emp.name, emp.title)

## Using Pandas

Pandas is a very handy way to interact with databased in Python, as it makes dumping and retrieving dataframes very easy.

In [31]:
import pandas as pd
pd.read_sql_query('SELECT * FROM stocks', conn)

,date,trans,symbol,qty,price
0,2006-01-05,BUY,RHAT,100.0,35.14


## SQLAlchemy

Summary of [this](https://www.youtube.com/watch?time_continue=1481&v=woKYyhLCcnU&feature=emb_logo) video.